# Dimensionality Reduction

* Reducing dimensionality does cause some information loss, just like compressing an image to JPEG can degrade its quality, so even though it will speed up training, it may make your system perform slightly worse. It also makes your pipelines a bit more complex and thus harder to maintain, so first try to train your system with the original data before considering using dimensionality reduction.

# A. The Curse of Dimensionality:

* Many Things behave differently in high-dimensional space.
    * If you pick a random point in a unit square (a 1x1 square), it will have only about 0.4% chance of being located less than 0.001 from the border.( it is very unlikely that a random point will be “extreme” along any dimension). If you pick a random point in a 10,000 dimensional unit hypercube, this probability is greater than 99.999999%. Most points in a high-dimensional hypercube are very close to the border.
    * If you pick 2 points randomly in a unit square, the distance between these 2 points will be on average, roughly 0.52. If you pick 2 random points in a 3D unit cube, the average distance will be roughly 0.66. In a 1,000,000 dimensional unit hypercube, the average distance will be about 408.25.

* Since there are plenty of space in high dimensions, high-dimensional datasets are at risk of being sparse,
    * Most training instances are likely to be far away from each other.
    * A new instance will likely be far away from any training instance, making predictions much less reliable than in lower dimensions, since they will be based on much larger extrapolations.
    * The more dimensions the training set has, the greater the risk of overfitting it.

* One solution to the curse of dimensionality is to increase the size of the training set to reach a sufficient density of training instances. Unfortunately, in practice, the number of training instances required to reach a given density grows exponentially with the number of dimensions.